In [0]:
#get the file name from the adf
fileName = dbutils.widgets.get('fileName')
#fileName = 'Product.csv'
fileNameWithoutExt = fileName.split('.')[0]
print(fileNameWithoutExt)

Product


In [0]:
import pyspark.sql.functions as F
#from datetime import datetme as dt

#Just change all the values here based on the resource name you have created in your environemnt and workspace.

sqlDbName = 'apmorgandb'
dbUserName = 'apmorganadmin'
passwordKey = 'sqldbpassword'
stgAccountSASTokenKey = 'sastokenstorage'
landingFileName =fileName #'Product'  #dbutils.widgets.get('Product')
databricksScopeName ='apmorgansecretscope'
dbServer = 'apmorganreyden'
dbServerPortNumber ='1433'
storageContainer ='apmorgandata'
storageAccount='apmorganproject1'
landingMountPoint ='/mnt'


In [0]:
if not any(mount.mountPoint == landingMountPoint for mount in dbutils.fs.mounts()):
    dbutils.fs.mount( source = 'wasbs://{}@{}.blob.core.windows.net'.format(storageContainer, storageAccount), mount_point= landingMountPoint, extra_configs ={'fs.azure.sas.{}.{}.blob.core.windows.net'.format(storageContainer,storageAccount):dbutils.secrets.get(scope = databricksScopeName, key= stgAccountSASTokenKey)})
    print('Mounted the storage account successfully')
else:
    print('Storage account already mounted')

Mounted the storage account successfully


In [0]:
#connect to Azure SQL DB
dbPassword = dbutils.secrets.get(scope = databricksScopeName, key= passwordKey)
serverurl = 'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer, dbServerPortNumber,sqlDbName,dbUserName)
connectionProperties = {
    'password':dbPassword,
    'driver':'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}
df = spark.read.jdbc(url = serverurl, table = 'dbo.FileDetailsFormat', properties= connectionProperties)
display(df)


FileNo,FileName,ColumnName,ColumnDateFormat,ColumnIsNull,ModifiedDate
1,Product,StartDate,MM-dd-yyyy,true,2012-06-18T22:34:09Z
1,Product,EndDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
1,Product,CreateDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
1,Product,ModifiedDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,ModifiedDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,StartDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,EndDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
3,CustomerDetail,CreateDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
3,CustomerDetail,ActiveDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z


In [0]:
df1 = spark.read.csv('/mnt/landingzone/'+fileName, inferSchema=True, header=True)
#display(df1)

# Rule
errorFlag=False
errorMessage = ''
totalcount = df1.count()
print(totalcount)
distinctCount = df1.distinct().count()
print(distinctCount)
if distinctCount !=totalcount:
    errorFlag = True
    errorMessage = 'Duplication Found. Rule 1 Failed'
print(errorMessage)
    
# Rule 2
df2 = df.filter(df.FileName==fileNameWithoutExt).select('ColumnName','ColumnDateFormat' )
rows = df2.collect()
for r in rows:
    colName = r[0]
    colFormat =r[1]
    print(colName, colFormat)
    #display(df1.filter(F.to_date(colName, colFormat).isNull() ==True))
    formatCount =df1.filter(F.to_date(colName, colFormat).isNotNull() ==True).count()
    if formatCount != totalcount:
        errorFlag = True
        errorMessage = errorMessage +' DateFormat is incorrect for {} '.format(colName)
    else:
        print('All rows are good for ', colName)
print(errorMessage)

if errorFlag:
    dbutils.fs.mv('/mnt/landingzone/'+fileName,'/mnt/rejected/'+fileName )
    dbutils.notebook.exit('{"errorFlag": "true", "errorMessage":"'+errorMessage +'"}')
else:
    dbutils.fs.mv('/mnt/landingzone/'+fileName,'/mnt/staging/'+fileName )
    print('{"errorFlag": "false", "errorMessage":"No error"}')

In [0]:
df3 = spark.read.jdbc(url = serverurl, table = 'dbo.Products', properties= connectionProperties)
display(df3)

ProductId,Price,guid,StartDate,EndDate,CreateDate,ModifiedDate


In [0]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("CSV_to_AzureSQL").getOrCreate()

# Read the CSV file into a Spark DataFrame
df3 = spark.read.csv("/mnt/staging/Product.csv", header=True, inferSchema=True)


In [0]:
from pyspark.sql.functions import col
df4 = df3.withColumn("Price", col("Price").cast("decimal(10,2)"))
df4.printSchema()

root
 |-- ProductId: string (nullable = true)
 |-- Price: decimal(10,2) (nullable = true)
 |-- guid: string (nullable = true)
 |-- StartDate: date (nullable = true)
 |-- EndDate: date (nullable = true)
 |-- CreateDate: date (nullable = true)
 |-- ModifiedDate: date (nullable = true)



In [0]:
df4.write.jdbc(url=serverurl, table="dbo.Products", mode="append", properties=connectionProperties)